In [ ]:
import yaml
import glob
import pandas as pd
import requests
from bs4 import BeautifulSoup
import json
import numpy as np

#API authorizations
from requests_oauthlib import OAuth2Session
from oauthlib.oauth2 import LegacyApplicationClient
import requests

## API Authorizations

In [ ]:
from requests_oauthlib import OAuth2Session
from oauthlib.oauth2 import LegacyApplicationClient
username='username'
password='password'
client_id='client_id'
client_secret='client_secret'
token_url='token_url'
api_pointer='http://your.url/api/v1/'


username='syst-superset-api'
password='}V(LuGpDs_M?.2=`'
client_id='superset-api'
client_secret='1d6f4fac-d580-4ca3-8e9b-8e636e0c998b'
token_url='https://sso.hometogo.rocks/auth/realms/hometogo/protocol/openid-connect/token'
api_pointer='http://superset.kube-dev.hometogo.rocks/api/v1/'

params={
  "order_column": "changed_on",
  "order_direction": "asc",
  "page": 0,
  "page_size": 2
}

oauth = OAuth2Session(client=LegacyApplicationClient(client_id=client_id))
token = oauth.fetch_token(token_url=token_url,
        username=username, password=password, client_id=client_id,
        client_secret=client_secret)

Example request

In [ ]:
#Applying filters on datasets
rison_request='dataset/?q=(page_size:100,order_column:changed_on_delta_humanized,order_direction:asc,filters:!((col:database,opr:rel_o_m,value:2)))'
request_body=api_pointer+rison_request
response = requests.get(
request_body, 
    headers={'Authorization': 'Bearer {}'.format(token['access_token'])},
    params=params
)
response=response.json()

## Swapping database for datasets

In [ ]:
token = oauth.fetch_token(token_url=token_url,
        username=username, password=password, client_id=client_id,
        client_secret=client_secret)
rison_request='dataset'
request_body=api_pointer+rison_request
response = requests.get(
request_body, 
    headers={'Authorization': 'Bearer {}'.format(token['access_token'])},
    params=params
)
response=response.json()

In [ ]:
for i in response['ids']:
    rison_request='dataset/'+str(i)
    request_body=api_pointer+rison_request

    # Data to be written 
    dictionary ={ 
      "database_id": 2 #parameter that will be set after the API call is executed
    } 

    # Serializing json  
    json_object = json.dumps(dictionary) 

    response = requests.put(
    request_body, 
        headers={'Authorization': 'Bearer {}'.format(token['access_token'])},
        json=dictionary
    )

In [ ]:
#Single example of a change 

rison_request='dataset/20'
request_body=api_pointer+rison_request

# Data to be written 
dictionary ={ 
  "database_id": 2
} 
      
# Serializing json  
json_object = json.dumps(dictionary) 

response = requests.put(
request_body, 
    headers={'Authorization': 'Bearer {}'.format(token['access_token'])},
    json=dictionary
)

In [ ]:
response.json()

## Refreshing columns for active physical datasets

In [ ]:
#Active datasets: not named archived
#Physical datasets: exported right away from the data base
import requests
rison_request='dataset/?q=(page_size:1,page:0,order_column:changed_on_delta_humanized,order_direction:asc,filters:!((col:table_name,opr:nct,value:archived),(col:sql,opr:dataset_is_null_or_empty,value:true)))'
request_body=api_pointer+rison_request
response = requests.get(
request_body, 
    headers={'Authorization': 'Bearer {}'.format(token['access_token'])},
    params=params
)
response=response.json()

In [ ]:
number_of_elemnts=response['count']
page_size=20
scans_needed=number_of_elemnts//page_size+1
elements=[]

In [ ]:
for i in range(0,scans_needed):
    page_number=str(i)
    rison_request=f'dataset/?q=(page_size:{page_size},page:{page_number},order_column:changed_on_delta_humanized,order_direction:asc,filters:!((col:table_name,opr:nct,value:archived),(col:sql,opr:dataset_is_null_or_empty,value:true)))'
    request_body=api_pointer+rison_request
    response = requests.get(
    request_body, 
        headers={'Authorization': 'Bearer {}'.format(token['access_token'])},
        params=params
    )
    response=response.json()
    elements.extend(response['ids'])

In [ ]:
#Remove duplicates
elements=list(dict.fromkeys(elements))

In [ ]:
for i in elements:
    element_number=str(i)
    rison_request=f'dataset/{element_number}/refresh'
    request_body=api_pointer+rison_request
    response = requests.put(
    request_body, 
        headers={'Authorization': 'Bearer {}'.format(token['access_token'])}
    )

## Swapping charts source

For a given dashboard

In [ ]:
dashboard_id=73 #dashboard whose charts will be swapped out

In [ ]:
access_token=token_request()

import requests
api_pointer='http://superset.kube-dev.hometogo.rocks/api/v1/'
rison_request=f'dashboard/{dashboard_id}/charts'
request_body=api_pointer+rison_request
response = requests.get(
request_body, 
    headers={'Authorization': 'Bearer {}'.format(access_token)},
    params=params
)
response=response.json()

In [ ]:
for i in range(0,len(response['result'])):
    slice_id=response['result'][i]['form_data']['slice_id']
    rison_request=f'chart/{slice_id}'
    request_body=api_pointer+rison_request
    response = requests.get(
    request_body, 
        headers={'Authorization': 'Bearer {}'.format(access_token)},
        params=params
    )
    r_charts=response.json()
    slice_name=r_charts['result']['slice_name']
    print(str(i)+' , '+str(slice_id) + ' , '+ slice_name)

In [ ]:
access_token=token_request()
for i in range(0,len(response['result'])):
    slice_id=response['result'][i]['form_data']['slice_id']
    rison_request=f'chart/{slice_id}'
    request_body=api_pointer+rison_request
    response = requests.get(
    request_body, 
        headers={'Authorization': 'Bearer {}'.format(access_token)},
        params=params
    )
    r_charts=response.json()
    slice_name=r_charts['result']['slice_name']
    swap_chart_dataset(access_token,slice_id=slice_id,destination_dataset=711)
    print(str(i)+' , '+str(slice_id) + ' , '+ slice_name+' already swapped')

For a given dataset

In [ ]:
def swap_chart_dataset(token, slice_id, destination_dataset, api_pointer=api_pointer):
    rison_request=f'chart/{slice_id}'
    request_body=api_pointer+rison_request
    # Data to be written 
    dictionary ={
      "datasource_id": destination_dataset,
      "datasource_type": "table"
    }
    # Serializing json  
    json_object = json.dumps(dictionary) 
    response = requests.put(
    request_body, 
        headers={'Authorization': 'Bearer {}'.format(token)},
        json=dictionary
    )

In [ ]:
for i in range(0, len(chart_list)):
    access_token=token_request()
    print(chart_list[i]['id'])
    swap_chart_dataset(token=access_token, slice_id=chart_list[i]['id'], destination_dataset=1)

## Add saved queries using the API

For this to work, the query needs to run a query in the production DB. Further info in this link: 
https://apache-superset.slack.com/archives/C015WAZL0KH/p1643719968410269

In [ ]:
def saved_queries_swap_owner(access_token,superset_owner_id, db_id, sql, query_tags, query_name,query_to_add_id, api_pointer):
    access_token=access_token
    #Defining parameters
    superset_id=superset_owner_id
    db_id=db_id
    sql=sql
    query_to_add_id=query_to_add_id
    schema='reporting'
    query_tags=query_tags
    name=name
    label=str(query_tags).replace(',', '_').lower()+'.'+str(query_name).lower()

    #Executing the function
    api_pointer=api_pointer
    rison_request='saved_query/'
    request_body=api_pointer+rison_request
    # Data to be written 
    dictionary ={
      "db_id": db_id,
      "description": "Old Id "+str(query_to_add_id),
      "label": label,
      "schema": schema,
      "sql": sql
    }
    # Serializing json  
    json_object = json.dumps(dictionary) 
    access_token=token_request()
    response = requests.post(
    request_body, 
        headers={'Authorization': 'Bearer {}'.format(access_token)},
        json=dictionary
    )
    #Saving the saved query id
    superset_saved_query_id=response.json()['id']
    superset_id=int(superset_id)
    transform_query='''UPDATE saved_query SET user_id = {0}, changed_by_fk= {0}, created_by_fk = {0} WHERE id={1};'''.format(superset_id, superset_saved_query_id)
    output=[superset_saved_query_id,transform_query]
    return output